<a href="https://colab.research.google.com/github/yasaminesmati/Coursera-Programming-for-Everybody-Getting-Started-with-Python/blob/main/machine-learning-with-python/regression/mlfirst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import all we need

In [73]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

read dataset

In [2]:
df = pd.read_csv("home.csv")
df.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33


drop usd price

In [3]:
df.drop('Price(USD)', inplace=True, axis=1)

removing big areas

In [12]:
df['Area'] = df['Area'].replace([' ',','],'',regex=True)

In [13]:
df['Area'].dtype

dtype('O')

In [15]:
df['Area']=df['Area'].astype(int)

In [20]:
df = df[df['Area'] < 2000]
df


,Area,Room,Parking,Warehouse,Elevator,Address,Price
0,63,1,True,True,True,Shahran,1.850000e+09
1,60,1,True,True,True,Shahran,1.850000e+09
2,79,2,True,True,True,Pardis,5.500000e+08
3,95,2,True,True,True,Shahrake Qods,9.025000e+08
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09
...,...,...,...,...,...,...,...
3474,86,2,True,True,True,Southern Janatabad,3.500000e+09
3475,83,2,True,True,True,Niavaran,6.800000e+09
3476,75,2,False,False,False,Parand,3.650000e+08
3477,105,2,True,True,True,Dorous,5.600000e+09


remove empty addresses

In [21]:
df = df.dropna(axis=0, subset=['Address'])
df

,Area,Room,Parking,Warehouse,Elevator,Address,Price
0,63,1,True,True,True,Shahran,1.850000e+09
1,60,1,True,True,True,Shahran,1.850000e+09
2,79,2,True,True,True,Pardis,5.500000e+08
3,95,2,True,True,True,Shahrake Qods,9.025000e+08
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09
...,...,...,...,...,...,...,...
3474,86,2,True,True,True,Southern Janatabad,3.500000e+09
3475,83,2,True,True,True,Niavaran,6.800000e+09
3476,75,2,False,False,False,Parand,3.650000e+08
3477,105,2,True,True,True,Dorous,5.600000e+09


encode parking (true=1,false=0)

In [22]:
label = LabelEncoder()
df.Parking = label.fit_transform(df.Parking)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


encode warehouse

In [23]:
label = LabelEncoder()
df.Warehouse = label.fit_transform(df.Warehouse)

encode elevator

In [24]:
label = LabelEncoder()
df.Elevator = label.fit_transform(df.Elevator)

encode address

In [25]:
label = LabelEncoder()
df.Address = label.fit_transform(df.Address)

we have a numeric dataset

In [26]:
df

,Area,Room,Parking,Warehouse,Elevator,Address,Price
0,63,1,1,1,1,156,1.850000e+09
1,60,1,1,1,1,156,1.850000e+09
2,79,2,1,1,1,117,5.500000e+08
3,95,2,1,1,1,152,9.025000e+08
4,123,2,1,1,1,150,7.000000e+09
...,...,...,...,...,...,...,...
3474,86,2,1,1,1,163,3.500000e+09
3475,83,2,1,1,1,105,6.800000e+09
3476,75,2,0,0,0,115,3.650000e+08
3477,105,2,1,1,1,39,5.600000e+09


In [27]:
df.describe()

,Area,Room,Parking,Warehouse,Elevator,Address,Price
count,3451.000000,3451.000000,3451.000000,3451.000000,3451.000000,3451.000000,3.451000e+03
mean,107.176181,2.081136,0.847580,0.915097,0.787308,104.977398,5.376033e+09
std,71.183260,0.760107,0.359479,0.278778,0.409271,50.330048,8.124787e+09
min,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.600000e+06
25%,69.000000,2.000000,1.000000,1.000000,1.000000,61.000000,1.419500e+09
50%,90.000000,2.000000,1.000000,1.000000,1.000000,117.000000,2.900000e+09
75%,120.000000,2.000000,1.000000,1.000000,1.000000,146.000000,6.000000e+09
max,1000.000000,5.000000,1.000000,1.000000,1.000000,191.000000,9.240000e+10


split train and test

In [39]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

using linear regresssion

In [91]:
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['Address','Area']])
y = np.asanyarray(train[['Price']])
regr.fit (x, y)


LinearRegression()

test the test data

In [92]:
x = np.asanyarray(test[['Address','Area']])
y = np.asanyarray(test[['Price']])
print('Variance score: %.2f' % regr.score(x, y))

Variance score: 0.43
